# 0.Jupyter常用快捷键

* **Shift + Enter** : 执行当前cell并选中下个cell
* **Shift + J 或 Shift + Down** : 选择下一个cell
* **Shift + K 或 Shift + Up** : 选中上一个cell
* **Shift + M** : 合并选中cell
* **Ctrl + Enter** : 执行下一个cell

* **Enter** ： 编辑模式

* **Esc** : 命令模式
* **Esc a** : 在上方新建cell
* **Esc b** : 在下方新建cell
* **Esc m** : 切换到markdown
* **Esc y** : 切换到code
* **Esc l** : 显示行数
* **Esc d d** : 删除选中的cell
* **Esc 0** : 收起、打开output


# 1.python正则库

In [2]:
# encoding: UTF-8
# py3默认编码utf-8，无需声明
import re
pattern = re.compile(r'hello.*\!')
match = pattern.match('hello, World! how are you?')
if match:
    print(match.group())

hello, World!


# 2.jieba中文处理

特点
* 支持三种分词模式：
 * 精确模式，试图将句子最精确地切开，适合文本分析；
 * 全模式，把句子中所有的可以成词的词语都扫描出来, 速度非常快，但是不能解决歧义；
 * 搜索引擎模式，在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词。
* 支持繁体分词
* 支持自定义词典

代码对 Python 2/3 均兼容

安装

* 全自动安装：easy_install jieba 或者pip install jieba / pip3 install jieba
* 半自动安装：先下载 http://pypi.python.org/pypi/jieba/ ，解压后运行 python setup.py install
* 手动安装：将 jieba 目录放置于当前目录或者 site-packages 目录

通过 import jieba 来引用

## 2.1.基本分词函数与用法

### jieba.cut()接受三个参数：

* 需要分词的字符串
* cut_all 是否采用全模式
* HMM 是否使用HMM（隐马）模型

### jieba.cut_for_search()方法接受两个参数：
* 需要分词的字符串
* 是否使用HMM模型
适合构建构建搜索引擎倒排索引

In [9]:
import jieba

seg_list = jieba.cut("我在学习自然语言处理", cut_all=True)

print("/".join(seg_list))

我/在/学习/自然/自然语言/语言/处理


## 2.2.用户自定义词典

1. 可以用jieba.load_userdict(file_name)加载用户词典
2. 少量词汇可以自己手动加载
 * add_word(word, freq=None, tag=None)和del_word(word)动态添加和删除
 * suggest_freq(segment, tune=True)可调节单个词语的词频，使其能（或不能）被分出来
 
注意：动态添加词语后分词，需要讲HMM关闭。

In [ ]:
import jieba



## 2.3.关键词提取

### 基于TF-IDF算法的关键词抽取

jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS={})


### 基于TextRank算法的关键词抽取

基于PageRank算法，数据量越大效果越好。


## 2.4.词性标注
